# Báo cáo tổng hợp: Phân tích hiệu năng 3 thuật toán học máy

Notebook này tổng hợp, phân tích và so sánh hiệu năng của 3 thuật toán đã sử dụng trong dự án Amazon Reviews Sentiment Analysis:
- Logistic Regression
- Random Forest
- Gradient Boosting

Dữ liệu phân tích được lấy từ các kết quả thực nghiệm trong notebook model_performance_analysis.

## 1. Import Required Libraries and Load Analysis Results

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob
import os
from datetime import datetime

# Load comprehensive performance data
reports_dir = "../src/reports/"
json_files = glob.glob(os.path.join(reports_dir, "*.json"))
reports_data = []
for file_path in sorted(json_files):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    data['file_info'] = {'filename': os.path.basename(file_path), 'filepath': file_path}
    reports_data.append(data)

def extract_comprehensive_data(reports_data):
    comprehensive_data = []
    for report in reports_data:
        timestamp = report['training_summary']['timestamp']
        filename = report['file_info']['filename']
        training_info = {
            'filename': filename,
            'timestamp': timestamp,
            'total_time_seconds': report['training_summary']['total_time_seconds'],
            'train_dataset_size': report['training_summary']['train_dataset_size'],
            'test_dataset_size': report['training_summary']['test_dataset_size']
        }
        for model_name, model_data in report['model_results'].items():
            row = training_info.copy()
            row.update({
                'model_type': model_name,
                'model_name_full': model_data['model_name'],
                'train_accuracy': model_data['train_accuracy'],
                'test_accuracy': model_data['test_accuracy'],
                'overfitting_score': model_data['overfitting_score'],
                'f1_macro': model_data['f1_macro'],
                'f1_weighted': model_data['f1_weighted'],
                'training_time': model_data['training_time'],
                'tfidf_max_features': model_data['tfidf_params']['max_features'],
                'tfidf_ngram_range': str(model_data['tfidf_params']['ngram_range']),
                'tfidf_min_df': model_data['tfidf_params']['min_df'],
                'tfidf_max_df': model_data['tfidf_params']['max_df'],
                'tfidf_sublinear_tf': model_data['tfidf_params']['sublinear_tf'],
            })
            if model_name == 'logistic_regression':
                row.update({
                    'lr_C': model_data['model_params']['C'],
                    'lr_max_iter': model_data['model_params']['max_iter'],
                    'lr_solver': model_data['model_params']['solver']
                })
            elif model_name == 'random_forest':
                row.update({
                    'rf_n_estimators': model_data['model_params']['n_estimators'],
                    'rf_max_depth': model_data['model_params']['max_depth'],
                    'rf_min_samples_split': model_data['model_params']['min_samples_split'],
                    'rf_max_features': model_data['model_params']['max_features']
                })
            elif model_name == 'gradient_boosting':
                row.update({
                    'gb_n_estimators': model_data['model_params']['n_estimators'],
                    'gb_learning_rate': model_data['model_params']['learning_rate'],
                    'gb_max_depth': model_data['model_params']['max_depth'],
                    'gb_subsample': model_data['model_params']['subsample']
                })
            comprehensive_data.append(row)
    return pd.DataFrame(comprehensive_data)

df_comprehensive = extract_comprehensive_data(reports_data)
df_comprehensive['datetime'] = pd.to_datetime(df_comprehensive['timestamp'])

KeyError: 'timestamp'

## 2. Algorithm Overview and Comparison

| Thuật toán             | Đặc điểm chính | Ưu điểm | Nhược điểm |
|-----------------------|----------------|---------|------------|
| Logistic Regression   | Tuyến tính, giải thích tốt, nhanh | Đơn giản, dễ triển khai, ít overfit | Hiệu năng thấp với dữ liệu phi tuyến |
| Random Forest         | Ensemble, nhiều cây quyết định | Chống overfit tốt, xử lý phi tuyến | Chậm, khó giải thích, tốn tài nguyên |
| Gradient Boosting     | Boosting, học tuần tự | Hiệu năng cao, tối ưu tốt | Dễ overfit, nhạy cảm tham số, chậm |

## 3. Logistic Regression Analysis and Results

- Phân tích ảnh hưởng tham số C, solver, max_iter
- Đánh giá độ hội tụ và độ ổn định
- Hiệu năng thực tế trên tập dữ liệu

In [ ]:
lr_data = df_comprehensive[df_comprehensive['model_type'] == 'logistic_regression']
plt.figure(figsize=(10,5))
sns.lineplot(data=lr_data, x='datetime', y='test_accuracy', marker='o')
plt.title('Logistic Regression - Test Accuracy Over Time')
plt.ylabel('Test Accuracy')
plt.xlabel('Time')
plt.show()

sns.boxplot(data=lr_data, x='lr_C', y='test_accuracy')
plt.title('Impact of C Parameter on Test Accuracy')
plt.show()

## 4. Random Forest Analysis and Results

- Phân tích hiệu ứng ensemble, tham số n_estimators, max_depth, min_samples_split
- Đánh giá feature importance và overfitting
- Biến động hiệu năng theo cấu hình

In [ ]:
rf_data = df_comprehensive[df_comprehensive['model_type'] == 'random_forest']
plt.figure(figsize=(10,5))
sns.lineplot(data=rf_data, x='datetime', y='test_accuracy', marker='o')
plt.title('Random Forest - Test Accuracy Over Time')
plt.ylabel('Test Accuracy')
plt.xlabel('Time')
plt.show()

sns.boxplot(data=rf_data, x='rf_n_estimators', y='test_accuracy')
plt.title('Impact of n_estimators on Test Accuracy')
plt.show()

## 5. Gradient Boosting Analysis and Results

- Phân tích ảnh hưởng learning_rate, số vòng boosting, độ sâu cây
- Đánh giá thời gian huấn luyện và hiệu năng
- Xác định tham số tối ưu

In [ ]:
gb_data = df_comprehensive[df_comprehensive['model_type'] == 'gradient_boosting']
plt.figure(figsize=(10,5))
sns.lineplot(data=gb_data, x='datetime', y='test_accuracy', marker='o')
plt.title('Gradient Boosting - Test Accuracy Over Time')
plt.ylabel('Test Accuracy')
plt.xlabel('Time')
plt.show()

sns.boxplot(data=gb_data, x='gb_learning_rate', y='test_accuracy')
plt.title('Impact of Learning Rate on Test Accuracy')
plt.show()

## 6. Cross-Algorithm Performance Comparison

So sánh trực tiếp các thuật toán về accuracy, F1-score, thời gian huấn luyện, độ ổn định và xu hướng overfitting.

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(data=df_comprehensive, x='model_type', y='test_accuracy')
plt.title('Test Accuracy Comparison Across Algorithms')
plt.show()

plt.figure(figsize=(10,6))
sns.boxplot(data=df_comprehensive, x='model_type', y='f1_macro')
plt.title('F1 Macro Score Comparison Across Algorithms')
plt.show()

## 7. Parameter Impact Analysis Across Algorithms

Phân tích ảnh hưởng của các tham số TF-IDF (max_features, ngram_range, min_df, max_df) lên từng thuật toán. Trực quan hóa bằng heatmap và correlation analysis.

In [ ]:
perf_pivot = df_comprehensive.pivot_table(values='test_accuracy', index='model_type', columns='tfidf_max_features', aggfunc='mean')
plt.figure(figsize=(12,6))
sns.heatmap(perf_pivot, annot=True, fmt='.3f', cmap='RdYlGn')
plt.title('Test Accuracy by Model & TF-IDF max_features')
plt.show()

## 8. Best Configuration Recommendations

Bảng tổng hợp các cấu hình tối ưu cho từng thuật toán dựa trên các tiêu chí: accuracy, F1-score, balanced performance.

In [ ]:
def get_best_configs(df):
    bests = {}
    for model in df['model_type'].unique():
        data = df[df['model_type'] == model]
        best_acc = data.loc[data['test_accuracy'].idxmax()]
        best_f1 = data.loc[data['f1_macro'].idxmax()]
        balanced = data.copy()
        balanced['balanced_score'] = balanced['test_accuracy'] - balanced['overfitting_score']
        best_bal = balanced.loc[balanced['balanced_score'].idxmax()]
        bests[model] = {
            'accuracy': best_acc,
            'f1': best_f1,
            'balanced': best_bal
        }
    return bests

best_configs = get_best_configs(df_comprehensive)
for model, configs in best_configs.items():
    print(f"\nModel: {model}")
    print(f"  Best Accuracy: {configs['accuracy']['test_accuracy']:.4f} | Params: {configs['accuracy']}")
    print(f"  Best F1: {configs['f1']['f1_macro']:.4f} | Params: {configs['f1']}")
    print(f"  Best Balanced: {configs['balanced']['balanced_score']:.4f} | Params: {configs['balanced']}")

## 9. Algorithm Selection Guidelines

- Nếu dữ liệu tuyến tính, cần giải thích: Logistic Regression
- Nếu dữ liệu phức tạp, nhiều đặc trưng, cần chống overfit: Random Forest
- Nếu cần tối ưu hóa hiệu năng, chấp nhận tuning tham số: Gradient Boosting

### Decision Tree for Algorithm Selection

```mermaid
graph TD;
    A[Start] --> B{Dữ liệu tuyến tính?}
    B -- Có --> C[Logistic Regression]
    B -- Không --> D{Cần chống overfit?}
    D -- Có --> E[Random Forest]
    D -- Không --> F[Gradient Boosting]
```

### Tổng kết
- Logistic Regression: Đơn giản, nhanh, phù hợp dữ liệu tuyến tính
- Random Forest: Chống overfit tốt, phù hợp dữ liệu phức tạp
- Gradient Boosting: Hiệu năng cao, cần tuning kỹ